## Lab 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

## First, looking at `accounts.py`

Let's review some python code made mostly by a hard-working Engineering Team.

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
from accounts import Account

In [3]:
account = Account.get("Ed")
account

Account(name='ed', balance=9920.842, strategy='', holdings={'AMZN': 2}, transactions=[1 shares of AMZN at 7.014 each., 1 shares of AMZN at 72.144 each.], portfolio_value_time_series=[('2025-07-14 19:22:54', 10020.986), ('2025-07-14 19:24:46', 10102.842), ('2025-07-14 19:24:46', 10078.842), ('2025-07-14 19:24:56', 10104.842)])

In [4]:
account.buy_shares("AMZN", 1, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 9910.822, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 1, "price": 7.014, "timestamp": "2025-07-14 19:22:54", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 1, "price": 72.144, "timestamp": "2025-07-14 19:24:46", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 1, "price": 10.02, "timestamp": "2025-07-18 21:32:37", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-07-14 19:22:54", 10020.986], ["2025-07-14 19:24:46", 10102.842], ["2025-07-14 19:24:46", 10078.842], ["2025-07-14 19:24:56", 10104.842], ["2025-07-18 21:32:37", 9934.822]], "total_portfolio_value": 9934.822, "total_profit_loss": -65.17799999999988}'

In [5]:
account.report()

'{"name": "ed", "balance": 9910.822, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 1, "price": 7.014, "timestamp": "2025-07-14 19:22:54", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 1, "price": 72.144, "timestamp": "2025-07-14 19:24:46", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 1, "price": 10.02, "timestamp": "2025-07-18 21:32:37", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-07-14 19:22:54", 10020.986], ["2025-07-14 19:24:46", 10102.842], ["2025-07-14 19:24:46", 10078.842], ["2025-07-14 19:24:56", 10104.842], ["2025-07-18 21:32:37", 9934.822], ["2025-07-18 21:32:39", 9991.822]], "total_portfolio_value": 9991.822, "total_profit_loss": -8.177999999999884}'

In [6]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 1,
  'price': 7.014,
  'timestamp': '2025-07-14 19:22:54',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 1,
  'price': 72.144,
  'timestamp': '2025-07-14 19:24:46',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 1,
  'price': 10.02,
  'timestamp': '2025-07-18 21:32:37',
  'rationale': 'Because this bookstore website looks promising'}]

## Now we make an MCP server: `accounts_server.py`

In [7]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

In [8]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
model = "gpt-4o-mini"

In [9]:


async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Your current balance is **$9,910.82**. 

Regarding your holdings, you have **3 shares** of **Amazon (AMZN)**.

OpenAI Agents SDK takes care of the MCP Client, but we can also make one ourselves

## See `accounts_client.py` for our MCP Client

In [10]:
from accounts_client import read_accounts_resource

context = await read_accounts_resource("ed")
print(context)

{"name": "ed", "balance": 9910.822, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 1, "price": 7.014, "timestamp": "2025-07-14 19:22:54", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 1, "price": 72.144, "timestamp": "2025-07-14 19:24:46", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 1, "price": 10.02, "timestamp": "2025-07-18 21:32:37", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-07-14 19:22:54", 10020.986], ["2025-07-14 19:24:46", 10102.842], ["2025-07-14 19:24:46", 10078.842], ["2025-07-14 19:24:56", 10104.842], ["2025-07-18 21:32:37", 9934.822], ["2025-07-18 21:32:39", 9991.822], ["2025-07-18 21:33:17", 9952.822]], "total_portfolio_value": 9952.822, "total_profit_loss": -47.177999999999884}
